In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner
from lark.ops import Sig2Spec, MixedSig2Spec
from lark.data import *

1.8.1
0.8.0a0+e4e171a


In [4]:
torch.cuda.set_device(1)
torch.cuda.current_device()

1

In [5]:
# get list of models
torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)

Downloading: "https://github.com/zhanghang1989/ResNeSt/archive/master.zip" to /home/koen/.cache/torch/hub/master.zip


['resnest101',
 'resnest200',
 'resnest269',
 'resnest50',
 'resnest50_fast_1s1x64d',
 'resnest50_fast_1s2x40d',
 'resnest50_fast_1s4x24d',
 'resnest50_fast_2s1x64d',
 'resnest50_fast_2s2x40d',
 'resnest50_fast_4s1x64d',
 'resnest50_fast_4s2x40d']

In [20]:
cfg = Config(
    noise_nsr_dbs = [30, 20, 10, 3],
    sites=['SSW'], 
    use_neptune=True, 
    log_batch_metrics=True,
    n_epochs=150, 
    bs=32, 
    lr=1e-3, 
    model='resnest50',
    scheduler='torch.optim.lr_scheduler.CosineAnnealingLR'
)

In [21]:
cfg.training_dataset_size

10752

In [22]:
prep = MixedSig2Spec(cfg)
main_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
posp = torch.nn.Linear(in_features=2048, 
                       out_features=len(cfg.labels), bias=True)
main_model.fc = posp
model = torch.nn.Sequential(prep, main_model)
model = model.cuda()


Using cache found in /home/koen/.cache/torch/hub/zhanghang1989_ResNeSt_master


In [23]:
lrn = Learner("resnest50-max", cfg, model)

In [24]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-73


  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

...

2021-05-06 00:34:56 epoch: 149 train loss: 0.067728 train f1: 0.778276 valid loss: 0.151208 valid f1: 0.186824


  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-06 00:37:16 epoch: 150 train loss: 0.069319 train f1: 0.775356 valid loss: 0.114349 valid f1: 0.277305


Unexpected error in ping thread.
Traceback (most recent call last):
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/neptune/internal/threads/ping_thread.py", line 37, in run
    self.__backend.ping_experiment(self.__experiment)
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/neptune/internal/api_clients/hosted_api_clients/hosted_alpha_leaderboard_api_client.py", line 382, in ping_experiment
    self.leaderboard_swagger_client.api.ping(experimentId=str(experiment.internal_id)).response().result
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/bravado/http_future.py", line 239, in response
    six.reraise(*sys.exc_info())
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/six.py", line 703, in reraise
    raise value
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/bravado/http_future.py", line 200, in response
    swagger_result = self._get_swagger_result(incoming_response)

In [25]:
lrn.scheduler.state_dict()

{'total_steps': 15120,
 '_schedule_phases': [{'end_step': 4535.0,
   'start_lr': 'initial_lr',
   'end_lr': 'max_lr',
   'start_momentum': 'max_momentum',
   'end_momentum': 'base_momentum'},
  {'end_step': 15119,
   'start_lr': 'max_lr',
   'end_lr': 'min_lr',
   'start_momentum': 'base_momentum',
   'end_momentum': 'max_momentum'}],
 'anneal_func': <bound method OneCycleLR._annealing_cos of <torch.optim.lr_scheduler.OneCycleLR object at 0x7fda88226f40>>,
 'cycle_momentum': True,
 'use_beta1': True,
 'base_lrs': [4e-05],
 'last_epoch': 15120,
 '_step_count': 15121,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [4.0220261377938495e-09]}

In [26]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,353,30845,1985,417,0.227156
2,0.2,293,32083,747,477,0.323757
3,0.3,257,32398,432,513,0.352296
4,0.4,229,32524,306,541,0.350958
5,0.5,198,32590,240,572,0.327815
6,0.6,169,32639,191,601,0.299115
7,0.7,146,32675,155,624,0.272642
8,0.8,124,32715,115,646,0.245788
9,0.9,95,32788,42,675,0.209482


In [28]:
lrn.load_checkpoint('best')

{'epoch': 8,
 'valid_loss': 0.08509646032593753,
 'valid_score': 0.3254593312740326}

In [29]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,443,30986,1844,327,0.289827
2,0.2,325,32016,814,445,0.340492
3,0.3,260,32386,444,510,0.352782
4,0.4,219,32568,262,551,0.350120
5,0.5,186,32643,187,584,0.325459
6,0.6,166,32691,139,604,0.308837
7,0.7,140,32744,86,630,0.281125
8,0.8,113,32800,30,657,0.247536
9,0.9,80,32820,10,690,0.186047


In [48]:
lrn.load_checkpoint('latest')

{'epoch': 14,
 'valid_loss': 0.09588292491083082,
 'valid_score': 0.32781457901000977}

In [49]:
lrn.scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(lrn.optimizer, T_0=10, T_mult=2, eta_min=0.01, last_epoch=-1)

In [50]:
cfg.n_epochs = 150
cfg.lr = 1e-5
tdl = TrainDataset(cfg).loader
lrn.tdl = tdl


In [ ]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-70


  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-05 18:16:23 epoch:   1 train loss: 0.158388 train f1: 0.277102 valid loss: 0.112025 valid f1: 0.081181


  0%|          | 0/336 [00:00<?, ?it/s]